# **GMRes: Método del Residuo Mínimo Generalizado**

---

# **Seccion 1: Discutamos en la pizarra!**
- Teorema de Cayley Hamilton.
- Aplicación del teorema para llegar a una nueva definición de $A^{-1}$.
- Subespacio de Krylov.
- Gmres usando el subespacio de krylov

---

# **Sección 2: Krylov genera una base mal condicionada**

Definmaos una matriz random de $n \times n$

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display

In [7]:
N = 2

# A = np.array([[0.9, 0.4], 
            #   [-0.4, 0.9]])
# b = np.array([1, 0])

np.random.seed(7)
A = np.random.rand(N, N)
b = np.random.rand(N, 1)  

def plot_krylov_vectors(k):
    plt.figure(figsize=(10, 8))
    origin = np.zeros(2)
    
    vecs = np.zeros((2, k+1))
    vecs[:, 0] = b.flatten()
    
    for i in range(1, k+1):
        vecs[:, i] = A @ vecs[:, i-1] 
    

    colors = plt.cm.magma(np.linspace(0, 1, k+1))
    labels = np.array([r'$b$'] + [r'$A^{' + str(i) + r'}b$' for i in range(1, k+1)])
    
    for i in range(k+1):
        alpha = 0.7 if i < k else 1.0
        plt.quiver(origin[0], origin[1], vecs[0, i], vecs[1, i],
                   scale=1, scale_units='xy', angles='xy',
                   color=colors[i], alpha=alpha, width=0.015,
                   label=labels[i])
    
    plt.axhline(0, color='gray', lw=0.5)
    plt.axvline(0, color='gray', lw=0.5)
    
    max_val = np.max(np.abs(vecs)) * 1.2
    min_val = -max_val
    plt.xlim(min_val, max_val)
    plt.ylim(min_val, max_val)
    
    plt.gca().set_aspect('equal')
    plt.grid(alpha=0.3)
    plt.title(f'Base de Krylov: $k={k}$', fontsize=14)
    

    cond_num = np.linalg.cond(vecs)
    
    plt.figtext(0.5, 0.01, f'κ(Q) = {cond_num:.3e}', 
                ha='center', fontsize=12, bbox=dict(facecolor='white', alpha=0.8))
    
    plt.legend(loc='upper right', bbox_to_anchor=(1.15, 1))
    plt.tight_layout()
    plt.show()

k_slider = widgets.IntSlider(value=1, min=1, max=30, step=1, description='k:')
widgets.interact(plot_krylov_vectors, k=k_slider)

interactive(children=(IntSlider(value=1, description='k:', max=30, min=1), Output()), _dom_classes=('widget-in…

<function __main__.plot_krylov_vectors(k)>

## **Sección 2.1: Breve recordatorio del númerod de condición de una matriz**

Defición mátematica del número de condición:

$$\kappa_2(A) = \|A\|_2 \cdot \|A^{-1}\|_2$$

$$
\kappa_2(A) = \frac{\sigma_{\text{max}}(A)}{\sigma_{\text{min}}(A)}
$$

El significado practico se puede descomponer de la siguiente manera:
- $\kappa \approx 1$: La matriz esta bien condicionada.
- $\kappa >> 1$: La matriz esta mal condicionada.
- $\kappa = \infty$: La matriz es singular.

In [3]:
A = np.array([[1, 1.0001], [1, 1]])
cond_num = np.linalg.cond(A)
print(f"Número de condición de A: {cond_num:.2e}")
b = np.array([2, 2])

# Sistema original
x_exact = np.linalg.solve(A, b)
print(f"Sistema original: Ax = b\nA:\n{A}\nb:\n{b}")
print(f"Solución exacta: {x_exact}")

# Sistema perturbado
b_pert = b + np.array([0.001, 0])
x_pert = np.linalg.solve(A, b_pert)
print(f"\nSistema perturbado: Ax = b + perturbación\nA:\n{A}\nb + perturbación:\n{b_pert}")
print(f"Solución perturbada: {x_pert}")
# Cálculo del error relativo
print("\nCálculo del error relativo:")
error_rel = np.linalg.norm(x_pert - x_exact) / np.linalg.norm(x_exact)
print(f"Error relativo: {error_rel:.2f}")

Número de condición de A: 4.00e+04
Sistema original: Ax = b
A:
[[1.     1.0001]
 [1.     1.    ]]
b:
[2 2]
Solución exacta: [ 2. -0.]

Sistema perturbado: Ax = b + perturbación
A:
[[1.     1.0001]
 [1.     1.    ]]
b + perturbación:
[2.001 2.   ]
Solución perturbada: [-8. 10.]

Cálculo del error relativo:
Error relativo: 7.07


Un número de condición grande implica que:
- Los vectores de la matriz son casi linealmente depedientes.
- Tenemos perdida en la presición de los cálculos.
- Los errores se amplifican.

De aquí surge la necesidad de hacer alguna transformación como esta:
$$
\mathbb{K}_k = \textit{span}\{b, Ab, A^2b, ... A^{k-1}b\} = \textit{span}\{q_1, q_2, ..., q_k\} 
$$

- $q_i$ es un vector ortonormal, norma 1 y ortogonal al resto

Para lograr la transformación aplicamos ortonormalización de Grand Schmidt modificada (Paso de Arnoldi)

---

# **Seccion 3: Discutamos en la pizarra!**
- Paso de Arnoldi

# **Sección 4: Código GMRes**

In [8]:
A = np.array([
    [1, 1, 0, ],
    [0, 1, 0, ],
    [1, 1, 1, ]
], dtype=float)

b = np.array([1, 0, 0], dtype=float)
# linea 1
x0 = np.zeros(3)
max_iter = 3

In [9]:

Q = []
H_list = []

# Linea 2: r0 = b - Ax0
r0 = b - A.dot(x0)
# Linea 3: q1 = r0 / ||r0||
q1 = r0 / np.linalg.norm(r0)
Q = [q1]

print("="*50)
print("INICIALIZACIÓN")
print(f"x0: {x0}")
print(f"r0 = b - A*x0: {r0}")
print(f"||r0|| = {np.linalg.norm(r0):.4f}")
print(f"q1 = r0/||r0||: {q1}\n")

H = np.zeros((max_iter+1, max_iter))

# Linea 4: "for k in range(1, m+1):"
for k in range(max_iter):
    print("="*50)
    print(f"ITERACIÓN k = {k+1}")
    print("-"*30)
    
    # Linea 5: y = A*q_k
    y = A.dot(Q[k])
    print(f"y = A * q_{k+1}:")
    print(y)
    
    # Lineas 6-8: Ortogonalización de Gram-Schmidt
    for j in range(k+1):
        H[j,k] = np.dot(Q[j], y)
        y = y - H[j,k] * Q[j]
        print(f"  j={j+1}: h_{j+1}{k+1} = {H[j,k]:.4f}, y_actualizado = {y}")
    
    # Paso 9: h_{k+1,k} = ||y||_2
    H[k+1,k] = np.linalg.norm(y)
    print(f"h_{k+2}{k+1} = ||y|| = {H[k+1,k]:.4f}")
    
    # Paso 10-11: Nuevo vector q_{k+1}
    if H[k+1,k] > 1e-10:
        q_next = y / H[k+1,k]
        Q.append(q_next)
        print(f"q_{k+2} = y / h_{k+2}{k+1} = {q_next}")
    else:
        print("Breakdown detectado!")
        break
    
    # Paso 12: Resolver problema de mínimos cuadrados
    H_k = H[:k+2, :k+1]
    e1 = np.zeros(k+2)
    e1[0] = np.linalg.norm(r0)
    
    # Resolver min ||βe1 - H_k c||
    c_k, _, _, _ = np.linalg.lstsq(H_k, e1, rcond=None)
    
    # Paso 13: Calcular solución aproximada
    Q_k = np.array(Q[:k+1]).T
    x_k = Q_k.dot(c_k) + x0
    
    print("\nResultados parciales:")
    print(f"Matriz H_{k+1}:")
    print(H_k)
    print(f"Vector c_k: {c_k}")
    print(f"Solución x_{k+1}: {x_k}")
    print(f"Residuo: {np.linalg.norm(b - A.dot(x_k)):.4f}\n")
    H_list.append(H_k.copy())

print("="*50)
print("RESULTADO FINAL")
print(f"Solución aproximada: {x_k}")
print(f"Solución exacta: {np.linalg.solve(A, b)}")

INICIALIZACIÓN
x0: [0. 0. 0.]
r0 = b - A*x0: [1. 0. 0.]
||r0|| = 1.0000
q1 = r0/||r0||: [1. 0. 0.]

ITERACIÓN k = 1
------------------------------
y = A * q_1:
[1. 0. 1.]
  j=1: h_11 = 1.0000, y_actualizado = [0. 0. 1.]
h_21 = ||y|| = 1.0000
q_2 = y / h_21 = [0. 0. 1.]

Resultados parciales:
Matriz H_1:
[[1.]
 [1.]]
Vector c_k: [0.5]
Solución x_1: [0.5 0.  0. ]
Residuo: 0.7071

ITERACIÓN k = 2
------------------------------
y = A * q_2:
[0. 0. 1.]
  j=1: h_12 = 0.0000, y_actualizado = [0. 0. 1.]
  j=2: h_22 = 1.0000, y_actualizado = [0. 0. 0.]
h_32 = ||y|| = 0.0000
Breakdown detectado!
RESULTADO FINAL
Solución aproximada: [0.5 0.  0. ]
Solución exacta: [ 1.  0. -1.]
